# PercePiano Replica Training (4-Fold Cross-Validation)

Train the PercePiano replica model using 4-fold piece-based cross-validation,
matching the methodology and hyperparameters from the PercePiano paper SOTA.

## Attribution

> **PercePiano: Piano Performance Evaluation Dataset with Multi-level Perceptual Features**  
> Park, Kim et al.  
> Nature Scientific Reports 2024  
> Paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC11450231/  
> GitHub: https://github.com/JonghoKimSNU/PercePiano

## Methodology

Following the exact approach from `m2pf_dataset_compositionfold.py`:

- **Piece-based splits**: All performances of the same piece stay in the same fold
- **Test set**: Select pieces randomly until reaching ~15% of SAMPLES (not pieces)
- **4-fold CV**: Remaining pieces distributed round-robin across folds
- **Per-fold normalization**: Stats computed from training folds only

## Hyperparameters (SOTA Configuration - R2 = 0.397)

These parameters match the published SOTA from `2_run_comp_multilevel_total.sh` and `han_bigger256_concat.yml`:

| Parameter | SOTA Value | Notes |
|-----------|------------|-------|
| input_size | 78 | SOTA uses 78 features (excludes section_tempo) |
| batch_size | 8 | From SOTA training script |
| learning_rate | 2.5e-5 | From SOTA training script |
| hidden_size | 256 | HAN encoder dimension |
| dropout | 0.2 | Regularization |
| augment_train | False | SOTA doesn't use key augmentation |
| max_epochs | 200 | Extended training window |
| early_stopping_patience | 40 | More patience for convergence |
| gradient_clip_val | 2.0 | From parser.py |

## Expected Results

- Target R2: 0.35-0.40 (matching published SOTA of 0.397)
- Training time: ~8-12 hours on T4, ~3-5 hours on A100 (all 4 folds)

## Step 1: Environment Setup

In [ ]:
# Check GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

: 

In [ ]:
# Install rclone
!curl -fsSL https://rclone.org/install.sh | sudo bash 2>&1 | grep -E "(successfully|already)" || echo "rclone installed"

In [ ]:
# Install uv and clone repository
!curl -LsSf https://astral.sh/uv/install.sh | sh

import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"

# Clone repository
if not os.path.exists('/tmp/crescendai'):
    !git clone https://github.com/Jai-Dhiman/crescendai.git /tmp/crescendai

%cd /tmp/crescendai/model
!git pull
!git log -1 --oneline

# Clone original PercePiano for comparison (needed for data diagnostics)
PERCEPIANO_PATH = '/tmp/crescendai/model/data/raw/PercePiano'
if not os.path.exists(PERCEPIANO_PATH):
    print("\nCloning original PercePiano repository...")
    !git clone https://github.com/JonghoKimSNU/PercePiano.git {PERCEPIANO_PATH}
else:
    print(f"\nPercePiano already present at {PERCEPIANO_PATH}")

# Install dependencies
!uv pip install --system -e .
!pip install tensorboard rich

import torch
import pytorch_lightning as pl
print(f"\nPyTorch: {torch.__version__}")
print(f"Lightning: {pl.__version__}")

## Step 2: Configure Paths and Check rclone

In [ ]:
import os
import subprocess
from pathlib import Path

# Paths
DATA_ROOT = Path('/tmp/percepiano_vnet_split')
CHECKPOINT_ROOT = Path('/tmp/checkpoints/percepiano_kfold')
LOG_ROOT = Path('/tmp/logs/percepiano_kfold')
GDRIVE_DATA_PATH = 'gdrive:crescendai_data/percepiano_vnet_split'
GDRIVE_CHECKPOINT_PATH = 'gdrive:crescendai_checkpoints/percepiano_kfold'

print("="*60)
print("PERCEPIANO REPLICA TRAINING (4-FOLD CV)")
print("="*60)

# Create directories
CHECKPOINT_ROOT.mkdir(parents=True, exist_ok=True)
LOG_ROOT.mkdir(parents=True, exist_ok=True)
DATA_ROOT.mkdir(parents=True, exist_ok=True)

# Check rclone
result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)

if 'gdrive:' in result.stdout:
    print("rclone 'gdrive' remote: CONFIGURED")
    RCLONE_AVAILABLE = True
else:
    print("rclone 'gdrive' remote: NOT CONFIGURED")
    print("Run 'rclone config' in terminal to set up Google Drive")
    RCLONE_AVAILABLE = False

print(f"\nData directory: {DATA_ROOT}")
print(f"Checkpoint directory: {CHECKPOINT_ROOT}")
print(f"Log directory: {LOG_ROOT}")

## Step 3: Download Data from Google Drive

In [ ]:
import subprocess

if not RCLONE_AVAILABLE:
    raise RuntimeError("rclone not configured. Run 'rclone config' first.")

# Download preprocessed data
print("Downloading preprocessed VirtuosoNet features from Google Drive...")
subprocess.run(
    ['rclone', 'copy', GDRIVE_DATA_PATH, str(DATA_ROOT), '--progress'],
    capture_output=False
)

# Verify data
print("\n" + "="*60)
print("DATA VERIFICATION")
print("="*60)

total_samples = 0
for split in ['train', 'val', 'test']:
    split_dir = DATA_ROOT / split
    if split_dir.exists():
        count = len(list(split_dir.glob('*.pkl')))
        total_samples += count
        print(f"  {split}: {count} samples")
    else:
        print(f"  {split}: MISSING!")

print(f"  Total: {total_samples} samples")

stat_file = DATA_ROOT / 'stat.pkl'
print(f"  stat.pkl: {'present' if stat_file.exists() else 'MISSING!'}")

fold_file = DATA_ROOT / 'fold_assignments.json'
print(f"  fold_assignments.json: {'present' if fold_file.exists() else 'will be created'}")

## Step 3b: Data Diagnostics (Priority 1)

Compare our data loading with original PercePiano to identify any mismatches.
This is critical for reproducing SOTA results.

In [ ]:
import pickle
import numpy as np
from pathlib import Path

print("="*70)
print("DATA DIAGNOSTICS - Comparing with PercePiano SOTA")
print("="*70)

# Load a sample file
sample_files = list((DATA_ROOT / 'train').glob('*.pkl'))[:5]
if not sample_files:
    raise RuntimeError("No training data found!")

print(f"\nFound {len(list((DATA_ROOT / 'train').glob('*.pkl')))} training samples")

# Inspect first sample structure
print("\n" + "-"*70)
print("SAMPLE DATA STRUCTURE")
print("-"*70)

with open(sample_files[0], 'rb') as f:
    sample = pickle.load(f)

print(f"Sample keys: {list(sample.keys())}")
print(f"Input shape: {sample['input'].shape}")
print(f"Scores shape: {sample['scores'].shape if 'scores' in sample else 'N/A'}")

if 'note_location' in sample:
    print(f"Note location keys: {list(sample['note_location'].keys())}")
    for k, v in sample['note_location'].items():
        print(f"  {k}: shape={np.array(v).shape}, range=[{np.min(v)}, {np.max(v)}]")

# Check input feature dimensions
input_dim = sample['input'].shape[-1]
print(f"\nInput dimension: {input_dim}")
if input_dim == 78:
    print("  [OK] Matches SOTA (78 features, excludes section_tempo)")
elif input_dim == 79:
    print("  [WARNING] Has 79 features (includes section_tempo) - SOTA uses 78")
elif input_dim == 83:
    print("  [INFO] Has 83 features (78 base + 5 unnorm)")
elif input_dim == 84:
    print("  [WARNING] Has 84 features - original VirtuosoNet config")
else:
    print(f"  [ERROR] Unexpected dimension: {input_dim}")

# Check target scores
if 'scores' in sample:
    scores = np.array(sample['scores'])
    print(f"\nTarget scores:")
    print(f"  Shape: {scores.shape}")
    print(f"  Range: [{scores.min():.4f}, {scores.max():.4f}]")
    print(f"  Mean: {scores.mean():.4f}, Std: {scores.std():.4f}")
    
    if scores.min() >= 0 and scores.max() <= 1:
        print("  [OK] Targets normalized to [0, 1]")
    else:
        print("  [ERROR] Targets NOT in [0, 1] range - needs normalization!")

In [ ]:
print("\n" + "-"*70)
print("FEATURE STATISTICS (Priority 2)")
print("-"*70)
print("Checking if our feature normalization matches original PercePiano...")

# Load stat.pkl if available
stat_file = DATA_ROOT / 'stat.pkl'
if stat_file.exists():
    with open(stat_file, 'rb') as f:
        stats = pickle.load(f)
    print(f"\nstat.pkl contents:")
    print(f"  Keys: {list(stats.keys())}")
    
    if 'mean' in stats and 'std' in stats:
        # Handle both dict and array formats
        if isinstance(stats['mean'], dict):
            mean = np.array(list(stats['mean'].values()))
            std = np.array(list(stats['std'].values()))
            print(f"  Format: dict with {len(stats['mean'])} features")
            print(f"  Feature names: {list(stats['mean'].keys())[:5]}...")
        else:
            mean = np.array(stats['mean'])
            std = np.array(stats['std'])
        print(f"  Mean shape: {mean.shape}, Std shape: {std.shape}")
        print(f"  Mean range: [{mean.min():.4f}, {mean.max():.4f}]")
        print(f"  Std range: [{std.min():.4f}, {std.max():.4f}]")
else:
    print("\n[WARNING] stat.pkl not found - normalization may be different")

# Compute actual feature statistics from training data
print("\nComputing actual feature statistics from training samples...")
all_features = []
for pkl_file in list((DATA_ROOT / 'train').glob('*.pkl'))[:50]:  # Sample 50 files
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)
    all_features.append(data['input'])

all_features = np.concatenate(all_features, axis=0)
print(f"Sampled features shape: {all_features.shape}")

# Compute statistics
actual_mean = all_features.mean(axis=0)
actual_std = all_features.std(axis=0)

print(f"\nFeature statistics (computed from data):")
print(f"  Mean range: [{actual_mean.min():.4f}, {actual_mean.max():.4f}]")
print(f"  Std range:  [{actual_std.min():.4f}, {actual_std.max():.4f}]")

# Check z-score normalization for first 8 features (should be ~0 mean, ~1 std)
z_scored_features = ['midi_pitch', 'duration', 'beat_importance', 'measure_length', 
                     'qpm_primo', 'following_rest', 'distance_from_abs_dynamic', 
                     'distance_from_recent_tempo']

print(f"\nZ-scored features check (first 8 should have mean~0, std~1):")
for i, name in enumerate(z_scored_features[:min(8, all_features.shape[1])]):
    m, s = actual_mean[i], actual_std[i]
    status = "[OK]" if abs(m) < 0.5 and 0.5 < s < 2.0 else "[CHECK]"
    print(f"  {i}: {name:30s} mean={m:+.4f}, std={s:.4f} {status}")

# Verify categorical features (should be 0/1 or small range)
print(f"\nOther features (indices 8-13):")
for i in range(8, min(14, all_features.shape[1])):
    m, s = actual_mean[i], actual_std[i]
    unique_count = len(np.unique(all_features[:100, i].round(2)))
    print(f"  {i}: mean={m:+.4f}, std={s:.4f}, ~{unique_count} unique values")

In [ ]:
print("\n" + "-"*70)
print("COMPARISON WITH ORIGINAL PERCEPIANO")
print("-"*70)

# Check if original PercePiano data is available for comparison
original_data_path = Path('/tmp/crescendai/model/data/raw/PercePiano')
if not original_data_path.exists():
    print("[INFO] Original PercePiano data not available for comparison")
    print("       Repository should be cloned in Step 1")
else:
    print(f"[OK] PercePiano repository found at {original_data_path}")
    
    # Check for the fold splitting code
    fold_script = original_data_path / 'virtuoso/virtuoso/pyScoreParser/m2pf_dataset_compositionfold.py'
    if fold_script.exists():
        print(f"[OK] Found original fold split code: m2pf_dataset_compositionfold.py")
        print("\n  Key methodology from original code:")
        print("  - Test set: select pieces until reaching 15% of SAMPLES")
        print("  - CV folds: round-robin distribution by piece index")
        print("  - Only pieces with >1 sample are eligible for test set")
    
    # Try to load original pickle files if any exist
    original_pkls = list(original_data_path.glob('**/*.pkl'))
    if original_pkls:
        print(f"\nFound {len(original_pkls)} original pickle files")
        
        # Load one and compare structure
        try:
            with open(original_pkls[0], 'rb') as f:
                orig_sample = pickle.load(f)
            
            print(f"\nOriginal sample structure ({original_pkls[0].name}):")
            if isinstance(orig_sample, dict):
                print(f"  Keys: {list(orig_sample.keys())}")
                for k, v in orig_sample.items():
                    if hasattr(v, 'shape'):
                        print(f"    {k}: shape={v.shape}")
                    elif isinstance(v, (list, np.ndarray)):
                        print(f"    {k}: len={len(v)}")
            elif isinstance(orig_sample, list):
                print(f"  List of {len(orig_sample)} items")
                if orig_sample and isinstance(orig_sample[0], dict):
                    print(f"  First item keys: {list(orig_sample[0].keys())}")
        except Exception as e:
            print(f"  Could not load sample: {e}")
    else:
        print("\n[INFO] No preprocessed pickle files in PercePiano repo")
        print("       (Original data requires running their preprocessing pipeline)")

# Check our data against expected SOTA format
print("\n" + "-"*70)
print("DATA FORMAT VALIDATION")
print("-"*70)

expected_keys = ['input', 'scores', 'note_location']
sample_keys = list(sample.keys())

print(f"Expected keys: {expected_keys}")
print(f"Our keys: {sample_keys}")

# Check for 'labels' key (used in our format instead of 'scores')
if 'labels' in sample_keys:
    print("[OK] 'labels' key present (mapped to 'scores' by dataset loader)")
elif 'scores' in sample_keys:
    print("[OK] 'scores' key present")
else:
    print("[ERROR] Neither 'labels' nor 'scores' key found!")

missing_keys = set(['input', 'note_location']) - set(sample_keys)
if not missing_keys:
    print("[OK] Required keys (input, note_location) present")
else:
    print(f"[ERROR] Missing keys: {missing_keys}")

extra_keys = set(sample_keys) - set(['input', 'note_location', 'labels', 'scores'])
if extra_keys:
    print(f"[INFO] Extra keys (OK): {extra_keys}")

# Validate note_location format
if 'note_location' in sample:
    nl = sample['note_location']
    expected_nl_keys = {'beat', 'measure', 'voice'}
    nl_keys = set(nl.keys())
    
    if expected_nl_keys.issubset(nl_keys):
        print("[OK] note_location has required keys (beat, measure, voice)")
    else:
        print(f"[ERROR] note_location missing: {expected_nl_keys - nl_keys}")

print("\n" + "="*70)
print("DATA DIAGNOSTICS COMPLETE")
print("="*70)

## Step 4: Create Fold Assignments

In [ ]:
from src.percepiano.data.kfold_split import (
    create_piece_based_folds,
    save_fold_assignments,
    load_fold_assignments,
    print_fold_statistics,
)

FOLD_FILE = DATA_ROOT / 'fold_assignments.json'
N_FOLDS = 4
TEST_RATIO = 0.15
SEED = 42

print("="*60)
print("FOLD ASSIGNMENT CREATION")
print("="*60)

# Force regeneration to use corrected methodology
# - Test set: select pieces until ~15% of SAMPLES (PercePiano methodology)
# - CV folds: greedy bin-packing for balanced sample counts (improvement over round-robin)
FORCE_REGENERATE = True

if FOLD_FILE.exists() and not FORCE_REGENERATE:
    print(f"\nLoading existing fold assignments from {FOLD_FILE}")
    fold_assignments = load_fold_assignments(FOLD_FILE)
else:
    if FOLD_FILE.exists():
        print(f"\nRemoving old fold assignments (regenerating with balanced methodology)...")
        FOLD_FILE.unlink()
    
    print(f"\nCreating new {N_FOLDS}-fold piece-based splits...")
    print("  Test set: select pieces until ~15% of SAMPLES")
    print("  CV folds: greedy bin-packing for balanced sample counts")
    fold_assignments = create_piece_based_folds(
        data_dir=DATA_ROOT,
        n_folds=N_FOLDS,
        test_ratio=TEST_RATIO,
        seed=SEED,
    )
    save_fold_assignments(fold_assignments, FOLD_FILE)

# Print statistics
print_fold_statistics(fold_assignments, n_folds=N_FOLDS)

## Step 5: Training Configuration (SOTA)

Configuration matched to published SOTA (R2 = 0.397):
- `input_size = 78` (excludes section_tempo)
- `learning_rate = 2.5e-5`
- `batch_size = 8`
- `augment_train = False`

In [ ]:
import torch
torch.set_float32_matmul_precision('medium')

# Enable better CUDA error reporting
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# PercePiano SOTA Configuration
# Hyperparameters matched to published SOTA (R2 = 0.397)
# Sources:
#   - Training script: 2_run_comp_multilevel_total.sh
#   - Config: han_bigger256_concat.yml
#   - Paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC11450231/
CONFIG = {
    # K-Fold settings
    'n_folds': N_FOLDS,
    'test_ratio': TEST_RATIO,
    
    # Data
    'data_dir': str(DATA_ROOT),
    'checkpoint_dir': str(CHECKPOINT_ROOT),
    'log_dir': str(LOG_ROOT),
    
    # Model input (SOTA uses 78 features, excludes section_tempo)
    'input_size': 78,
    
    # HAN Architecture (han_bigger256_concat.yml)
    'hidden_size': 256,
    'note_layers': 2,
    'voice_layers': 2,
    'beat_layers': 2,
    'measure_layers': 1,
    'num_attention_heads': 8,
    'final_hidden': 128,
    
    # Training - MATCHED TO SOTA (2_run_comp_multilevel_total.sh)
    'learning_rate': 2.5e-5,  # SOTA value
    'weight_decay': 1e-5,
    'dropout': 0.2,
    'batch_size': 8,          # SOTA value
    'max_epochs': 200,
    'early_stopping_patience': 40,
    'gradient_clip_val': 2.0,
    'precision': '16-mixed',
    
    # Dataset
    'max_notes': 1024,
    'num_workers': 0,  # Avoid shared memory issues on Thunder Compute
    'augment_train': False,  # SOTA doesn't use augmentation
}

print("="*60)
print("TRAINING CONFIGURATION (SOTA)")
print("="*60)
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

# Print training dynamics info
steps_per_epoch = 477 // CONFIG['batch_size']  # approximate
lr_decay_epoch = 3000 // steps_per_epoch if steps_per_epoch > 0 else 999
print(f"\nTraining dynamics:")
print(f"  Steps per epoch (approx): {steps_per_epoch}")
print(f"  LR decay (StepLR) at epoch: ~{lr_decay_epoch}")
print(f"  LR after decay: {CONFIG['learning_rate'] * 0.98:.2e}")

## Step 6: Initialize K-Fold Trainer

In [ ]:
from src.percepiano.training.kfold_trainer import KFoldTrainer
import pytorch_lightning as pl

# Set seed for reproducibility
pl.seed_everything(42, workers=True)

# Create K-Fold trainer
kfold_trainer = KFoldTrainer(
    config=CONFIG,
    fold_assignments=fold_assignments,
    data_dir=DATA_ROOT,
    checkpoint_dir=CHECKPOINT_ROOT,
    log_dir=LOG_ROOT,
    n_folds=N_FOLDS,
)

print("K-Fold Trainer initialized")
print(f"  Folds: {N_FOLDS}")
print(f"  Checkpoints: {CHECKPOINT_ROOT}")
print(f"  Logs: {LOG_ROOT}")

## Step 7: Train All Folds

In [ ]:
print("="*60)
print("STARTING 4-FOLD CROSS-VALIDATION TRAINING")
print("="*60)
print("\nPercePiano SOTA baselines:")
print("  Bi-LSTM: R2 = 0.185")
print("  MidiBERT: R2 = 0.313")
print("  Bi-LSTM + SA + HAN: R2 = 0.397 (SOTA)")
print("="*60)

# Train all folds
aggregate_metrics = kfold_trainer.train_all_folds(verbose=True)

# Save results
kfold_trainer.save_results()

## Step 7b: Training Diagnostics

Analyze prediction statistics to identify collapsed dimensions or other issues.

In [ ]:
import torch
import numpy as np
from src.percepiano.models.percepiano_replica import PERCEPIANO_DIMENSIONS

print("="*70)
print("TRAINING DIAGNOSTICS - Prediction Analysis")
print("="*70)

# Get the best model from fold 0 for analysis
fold_0_ckpt = list(CHECKPOINT_ROOT.glob('fold_0/*.ckpt'))
if not fold_0_ckpt:
    print("[WARNING] No fold 0 checkpoint found - run training first")
else:
    from src.percepiano.models.percepiano_replica import PercePianoVNetModule
    from src.percepiano.data.percepiano_vnet_dataset import PercePianoVNetDataset
    from torch.utils.data import DataLoader
    
    # Load model
    model = PercePianoVNetModule.load_from_checkpoint(fold_0_ckpt[0])
    model.eval()
    model.cuda() if torch.cuda.is_available() else model.cpu()
    
    # Load validation data for fold 0
    val_dataset = PercePianoVNetDataset(
        data_dir=DATA_ROOT,
        split='val',
        max_notes=CONFIG['max_notes'],
    )
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # Collect predictions
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for batch in val_loader:
            # Move to device
            device = next(model.parameters()).device
            input_features = batch['input_features'].to(device)
            note_locations = {
                'beat': batch['note_locations_beat'].to(device),
                'measure': batch['note_locations_measure'].to(device),
                'voice': batch['note_locations_voice'].to(device),
            }
            
            outputs = model(input_features, note_locations)
            all_preds.append(outputs['predictions'].cpu())
            all_targets.append(batch['scores'])
    
    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()
    
    # Analyze predictions
    print("\n" + "-"*70)
    print("PREDICTION STATISTICS")
    print("-"*70)
    
    print(f"\nOverall:")
    print(f"  Predictions - mean: {all_preds.mean():.4f}, std: {all_preds.std():.4f}")
    print(f"  Predictions - range: [{all_preds.min():.4f}, {all_preds.max():.4f}]")
    print(f"  Targets     - mean: {all_targets.mean():.4f}, std: {all_targets.std():.4f}")
    print(f"  Targets     - range: [{all_targets.min():.4f}, {all_targets.max():.4f}]")
    
    # Per-dimension analysis
    print("\n" + "-"*70)
    print("PER-DIMENSION ANALYSIS (Collapsed = std < 0.05)")
    print("-"*70)
    
    collapsed_dims = []
    print(f"\n{'Dimension':<25} {'Pred Mean':>10} {'Pred Std':>10} {'Tgt Std':>10} {'Status':<12}")
    print(f"{'-'*25} {'-'*10} {'-'*10} {'-'*10} {'-'*12}")
    
    for i, dim in enumerate(PERCEPIANO_DIMENSIONS):
        pred_mean = all_preds[:, i].mean()
        pred_std = all_preds[:, i].std()
        tgt_std = all_targets[:, i].std()
        
        if pred_std < 0.05:
            status = "[COLLAPSED]"
            collapsed_dims.append(dim)
        elif pred_std < tgt_std * 0.5:
            status = "[NARROW]"
        else:
            status = "[OK]"
        
        print(f"{dim:<25} {pred_mean:>10.4f} {pred_std:>10.4f} {tgt_std:>10.4f} {status:<12}")
    
    print(f"\n{'-'*70}")
    print(f"SUMMARY: {len(collapsed_dims)}/19 dimensions collapsed")
    if collapsed_dims:
        print(f"Collapsed: {', '.join(collapsed_dims)}")
    
    # Check if sigmoid is working
    print("\n" + "-"*70)
    print("SIGMOID CHECK")
    print("-"*70)
    
    if all_preds.min() >= 0 and all_preds.max() <= 1:
        print("[OK] Predictions bounded in [0, 1] - sigmoid working")
    else:
        print(f"[ERROR] Predictions outside [0, 1]: [{all_preds.min():.4f}, {all_preds.max():.4f}]")
        print("        Check if sigmoid is applied in forward pass!")

print("\n" + "="*70)
print("DIAGNOSTICS COMPLETE")
print("="*70)

## Priority 3: Sequence Iteration Gap

The SOTA config uses `sequence_iteration: 3` which iteratively refines the hierarchy.
Our current implementation does a single pass. This may contribute to the R2 gap.

**From original `encoder_score.py:33`:**
```python
for i in range(self.num_sequence_iteration):
    # Refine hierarchy representations
```

**Implementation TODO:**
- Add `sequence_iterations` parameter to `PercePianoHAN`
- Wrap hierarchy processing in iteration loop
- Each iteration feeds measure context back to note level

In [ ]:
# Sync checkpoints to Google Drive after training
if RCLONE_AVAILABLE:
    print("Syncing checkpoints to Google Drive...")
    subprocess.run(
        ['rclone', 'copy', str(CHECKPOINT_ROOT), GDRIVE_CHECKPOINT_PATH, '--progress'],
        capture_output=False
    )
    print("Sync complete!")

## Step 8: Test Set Evaluation

In [ ]:
# Evaluate all fold models on held-out test set
test_results = kfold_trainer.evaluate_on_test(verbose=True)

## Step 9: Per-Fold Results Summary

In [ ]:
import numpy as np
from src.percepiano.models.percepiano_replica import PERCEPIANO_DIMENSIONS

print("="*80)
print("PER-FOLD VALIDATION RESULTS")
print("="*80)

print(f"\n{'Fold':<6} {'Val R2':>10} {'Val Pearson':>12} {'Val MAE':>10} {'Val RMSE':>10} {'Epochs':>8} {'Time (s)':>10}")
print(f"{'-'*6} {'-'*10} {'-'*12} {'-'*10} {'-'*10} {'-'*8} {'-'*10}")

for m in kfold_trainer.fold_metrics:
    print(f"{m.fold_id:<6} {m.val_r2:>+10.4f} {m.val_pearson:>+12.4f} {m.val_mae:>10.4f} {m.val_rmse:>10.4f} {m.epochs_trained:>8} {m.training_time_seconds:>10.1f}")

print(f"{'-'*6} {'-'*10} {'-'*12} {'-'*10} {'-'*10} {'-'*8} {'-'*10}")
print(f"{'Mean':<6} {aggregate_metrics.mean_r2:>+10.4f} {aggregate_metrics.mean_pearson:>+12.4f} {aggregate_metrics.mean_mae:>10.4f} {aggregate_metrics.mean_rmse:>10.4f}")
print(f"{'Std':<6} {aggregate_metrics.std_r2:>+10.4f} {aggregate_metrics.std_pearson:>+12.4f} {aggregate_metrics.std_mae:>10.4f} {aggregate_metrics.std_rmse:>10.4f}")

## Step 10: Per-Dimension Analysis

In [ ]:
print("="*80)
print("PER-DIMENSION R2 (Mean +/- Std across folds)")
print("="*80)

# Sort dimensions by mean R2
sorted_dims = sorted(
    aggregate_metrics.per_dim_mean_r2.items(),
    key=lambda x: x[1],
    reverse=True
)

print(f"\n{'Dimension':<25} {'Mean R2':>10} {'Std R2':>10} {'Status':<12}")
print(f"{'-'*25} {'-'*10} {'-'*10} {'-'*12}")

for dim, mean_r2 in sorted_dims:
    std_r2 = aggregate_metrics.per_dim_std_r2[dim]
    
    if mean_r2 >= 0.3:
        status = "[GOOD]"
    elif mean_r2 >= 0.1:
        status = "[OK]"
    elif mean_r2 >= 0:
        status = "[WEAK]"
    else:
        status = "[FAILED]"
    
    print(f"{dim:<25} {mean_r2:>+10.4f} {std_r2:>10.4f} {status:<12}")

# Summary
positive = sum(1 for d, r in sorted_dims if r > 0)
strong = sum(1 for d, r in sorted_dims if r >= 0.2)
n_dims = len(sorted_dims)

print(f"\nSummary: {positive}/{n_dims} positive R2, {strong}/{n_dims} strong (>= 0.2)")

## Step 11: Final Summary and Save

In [ ]:
import json
import torch
from pathlib import Path

print("="*80)
print("FINAL SUMMARY")
print("="*80)

# Cross-validation results
print(f"\n4-Fold Cross-Validation Results:")
print(f"  Mean R2:       {aggregate_metrics.mean_r2:.4f} +/- {aggregate_metrics.std_r2:.4f}")
print(f"  Mean Pearson:  {aggregate_metrics.mean_pearson:.4f} +/- {aggregate_metrics.std_pearson:.4f}")
print(f"  Mean Spearman: {aggregate_metrics.mean_spearman:.4f} +/- {aggregate_metrics.std_spearman:.4f}")
print(f"  Mean MAE:      {aggregate_metrics.mean_mae:.4f} +/- {aggregate_metrics.std_mae:.4f}")
print(f"  Mean RMSE:     {aggregate_metrics.mean_rmse:.4f} +/- {aggregate_metrics.std_rmse:.4f}")
print(f"  Training time: {aggregate_metrics.total_training_time/60:.1f} minutes")

# Test set results
print(f"\nTest Set (Ensemble of 4 models):")
print(f"  R2:       {test_results['ensemble']['r2']:.4f}")
print(f"  Pearson:  {test_results['ensemble']['pearson']:.4f}")
print(f"  Spearman: {test_results['ensemble']['spearman']:.4f}")
print(f"  MAE:      {test_results['ensemble']['mae']:.4f}")
print(f"  RMSE:     {test_results['ensemble']['rmse']:.4f}")

# Comparison to baselines
print(f"\nComparison to PercePiano baselines:")
print(f"  Bi-LSTM:      R2 = 0.185")
print(f"  MidiBERT:     R2 = 0.313")
print(f"  HAN SOTA:     R2 = 0.397")
print(f"  Ours (CV):    R2 = {aggregate_metrics.mean_r2:.3f} +/- {aggregate_metrics.std_r2:.3f}")
print(f"  Ours (Test):  R2 = {test_results['ensemble']['r2']:.3f}")

# Interpretation
cv_r2 = aggregate_metrics.mean_r2
test_r2 = test_results['ensemble']['r2']

print(f"\nInterpretation:")
if cv_r2 >= 0.35:
    print(f"  [EXCELLENT] CV R2 >= 0.35 matches published SOTA!")
elif cv_r2 >= 0.25:
    print(f"  [GOOD] CV R2 >= 0.25 is usable for pseudo-labeling")
elif cv_r2 >= 0.10:
    print(f"  [FAIR] CV R2 >= 0.10 shows learning, needs improvement")
else:
    print(f"  [NEEDS WORK] CV R2 < 0.10, significant improvement needed")

# Save ensemble model if good enough
if cv_r2 >= 0.25:
    print(f"\nModel qualifies for pseudo-labeling MAESTRO!")

In [ ]:
# Final sync to Google Drive
print("="*60)
print("SYNC TO GOOGLE DRIVE")
print("="*60)

if RCLONE_AVAILABLE:
    print(f"\nSyncing all checkpoints and results...")
    subprocess.run(
        ['rclone', 'copy', str(CHECKPOINT_ROOT), GDRIVE_CHECKPOINT_PATH, '--progress'],
        capture_output=False
    )
    
    # Also sync fold assignments back to data directory
    subprocess.run(
        ['rclone', 'copy', str(FOLD_FILE), GDRIVE_DATA_PATH, '--progress'],
        capture_output=False
    )
    
    print(f"\nSync complete!")
    print(f"  Checkpoints: {GDRIVE_CHECKPOINT_PATH}")
    print(f"  Fold assignments: {GDRIVE_DATA_PATH}")
else:
    print(f"\nrclone not available - skipping sync")

print(f"\n{'='*60}")
print("TRAINING COMPLETE")
print(f"{'='*60}")